In [39]:
import tensorflow as tf

from tensorflow.keras import layers, models

In [4]:
tf.__version__

'2.16.2'

In [5]:
model = tf.keras.Sequential()

In [13]:
def BasicBlock(x, filters, stride = 1):
    shortcut = x
    x = layers.Conv2D(filters, kernel_size = 3, strides = stride, padding = "same", kernel_initializer = "he_normal")(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    x = layers.Conv2D(filters, kernel_size = 3, strides = 1, padding = "same", kernel_initializer = "he_normal")(x)
    x = layers.BatchNormalization()(x)

    if stride != 1 or shortcut.shape[-1] != filters:
        shortcut = layers.Conv2D(filters, 1, strides = stride, kernel_initializer = "he_normal")(shortcut)
        shortcut = layers.BatchNormalization()(shortcut)
    x = layers.Add()([x + shortcut])
    x = layers.BatchNormalization()(x)
    return x

In [37]:
def make_layers(x, filters, blocks, stride = 1):
    x = BasicBlock(x, filters, stride = stride)
    for _ in range(1, blocks):
        x = BasicBlock(x, filters, stride = 1)
    return x

In [42]:
def ResNet18(input_shape = (32, 32, 3), num_classes = 10):
    inputs = layers.Input(shape = input_shape)

    x = layers.Conv2D(64, 3, strides = 1, padding = "same", kernel_initializer = "he_normal")(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)

    x = make_layers(x, filters = 64, blocks = 2, stride = 1)
    x = make_layers(x, filters = 128, blocks = 2, stride = 2)
    x = make_layers(x, filters = 256, blocks = 2, stride = 2)
    x = make_layers(x, filters = 512, blocks = 2, stride = 2)

    x = layers.GlobalAveragePooling2D()(x)
    outputs = layers.Dense(num_classes, activation = "softmax")(x)
    model = models.Model(inputs, outputs)
    return model

In [43]:
model = ResNet18()

In [45]:
model.compile(
    optimizer = "adam",
    loss = "categorical_crossentropy",
    metrics = ["accuracy"]
)